Stats descriptives des bases de données pour une première analyse

1) Importation des bases que l'on va utiliser

In [2]:
import pandas as pd

# Importer 3 bases CSV
df_voit = pd.read_csv("../Bases/q_voitvul_public_V3.csv", delimiter=";",encoding="ISO-8859-1")  # Modifie le délimiteur si nécessaire
df_tcmmen = pd.read_csv("../Bases/tcm_men_public_V3.csv", delimiter=",",encoding="ISO-8859-1")  # Modifie le délimiteur si nécessaire
df_men = pd.read_csv("../Bases/q_menage_public_V3.csv", delimiter=";",encoding="ISO-8859-1")  # Modifie le délimiteur si nécessaire

# Importer la base Excel
df_excel = pd.read_excel("../Bases/parc_vp_region_2024_1.xlsx", engine="openpyxl")  # Spécifie openpyxl pour les fichiers .xlsx
print(df_excel.head())  # Affiche les 5 premières lignes
#excel petit soucis à regler pour l'ouverture

/tmp/ipykernel_17367/4252442764.py:5: DtypeWarning: Columns (63) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tcmmen = pd.read_csv("../Bases/tcm_men_public_V3.csv", delimiter=",",encoding="ISO-8859-1")  # Modifie le délimiteur si nécessaire


  Parc par région des voitures particulières au 1er janvier, selon le carburant, l'âge, le statut, la vignette crit'air  \
0                              Source : SDES, Rsvero                                                                      
1                                                NaN                                                                      
2                                Région de résidence                                                                      
3                                           Inconnue                                                                      
4                                           Inconnue                                                                      

                 Unnamed: 1                       Unnamed: 2 Unnamed: 3  \
0                       NaN                              NaN        NaN   
1                       NaN                              NaN        NaN   
2  Code région de résidence         

In [3]:
#Affiche table voiturevul
print(df_voit.head())  # Affiche les 5 premières lignes

      IDENT_NUMVEH      IDENT_MEN  NUM_VEH     pond_veh  KVROUL  \
0  110000011400001  1100000114000        1  1842.977077       1   
1  110000011500001  1100000115000        1  1948.382441       1   
2  110000011500002  1100000115000        2  1948.382441       1   
3  110000011600001  1100000116000        1   833.453670       1   
4  110000012400001  1100000124000        1  1698.142782       1   

   cylindree_fin energie_fin  energie_agrege hybride  energie_statinfo  ...  \
0            1.6          ES             1.0     NaN               2.0  ...   
1            1.6          GO             2.0     NaN               4.0  ...   
2            1.4          ES             1.0     NaN               2.0  ...   
3            1.6          ES             1.0     NaN               2.0  ...   
4            1.5          GO             2.0     NaN               4.0  ...   

   KVKMVT      KVKM1ANV  indic_km_annu_corr KVKM1ANVT  KVCONS KVGAREJOUR  \
0     4.0   2478.916193             INTERPO   

2. Analyse des bases chargées  
2.1 Analyse de la base VoitVul : 

In [4]:
#Convertir les variables identifiants voitures et ménages en string
df_voit[["IDENT_NUMVEH", "IDENT_MEN"]] = df_voit[["IDENT_NUMVEH", "IDENT_MEN"]].astype(str)
#Création de nouvelles variables energies 
df_voit["energie_groupe"] = df_voit["energie_agrege"].apply(lambda x: 
                                           "Thermique" if x in [1, 2] else 
                                           "Electrique" if x == 5  else 
                                           "Hybride" if x == 4 else 
                                           "Autres")
print(df_voit.columns)
#Statistiques sur certaines variables surement
df_voit[["critair"]].describe()


Index(['IDENT_NUMVEH', 'IDENT_MEN', 'NUM_VEH', 'pond_veh', 'KVROUL',
       'cylindree_fin', 'energie_fin', 'energie_agrege', 'hybride',
       'energie_statinfo', 'energie_CritAir', 'critair', 'puis_fisc_fin',
       'ANNEE_1mec', 'age', 'ANNEE_acq', 'dur_acq', 'KVAQUIV', 'KVAQUINO01',
       'KVAQUINO02', 'KVAQUINO03', 'KVAQUINO04', 'KVAQUINO05', 'KVAQUINO06',
       'KVAQUINO07', 'KVAQUINO08', 'KVAQUINO09', 'KVAQUINO10', 'KVAQUINO11',
       'KVAQUINO12', 'KVAQUINO13', 'KVAQUINO14', 'KVAQUINO15', 'KVAQUINO16',
       'KVAQUINO17', 'KVAQUINO18', 'KVAQUINO19', 'KVAQUINO20', 'KVAQUINO21',
       'KVQUICONV01', 'KVQUICONV02', 'KVQUICONV03', 'KVQUICONV04',
       'KVQUICONV05', 'KVQUICONV06', 'KVQUICONV07', 'KVQUICONV08',
       'KVQUICONV09', 'KVQUICONV10', 'KVQUICONV11', 'KVQUICONV12',
       'KVQUICONV13', 'KVQUICONV14', 'KVQUICONV15', 'KVQUICONV16',
       'KVQUICONV17', 'KVQUICONV18', 'KVQUICONV19', 'KVQUICONV20',
       'KVQUICONV21', 'KVQUI9COV', 'KVQUICONPV', 'KVKMV', 'indic_km_c

,critair
count,17933
unique,7
top,2
freq,5840


    2.2 Analyse de la base tcm_men : 

In [5]:
#Affiche table tcm_menage
print(df_tcmmen.head())

       ident_men     ident_log  vague_enq  BS    pond_menC  NPERS  NCOUPLES  \
0  1100000114000  110000011400          1   0  1842.977077      1         0   
1  1100000115000  110000011500          1   0  1948.382441      4         1   
2  1100000116000  110000011600          1   0   833.453670      2         1   
3  1100000124000  110000012400          2   0  1698.142782      2         1   
4  1100000125000  110000012500          2   0  1994.633496      2         1   

   NENFANTS  NACTIFS  NENFACT  ...  pop_10min_pieton_train_res  \
0         0        0        0  ...                          80   
1         2        2        0  ...                          80   
2         0        0        0  ...                           0   
3         0        2        0  ...                          50   
4         0        1        0  ...                          50   

   pop_10min_voiture_tram_metro_res  pop_10min_voiture_train_res  \
0                                 0                         

In [15]:
#Convertir les variables identifiants voitures et ménages en string
df_tcmmen[["ident_log", "ident_men"]] = df_tcmmen[["ident_log", "ident_men"]].astype(str)


In [22]:
#Vérifier que la somme des pondérations des ménages vaut le nombre total de ménages en France
tot_pond_menC=df_tcmmen[["pond_menC"]].sum()
print(tot_pond_menC)

#Nombre moyen de personnes dans un ménage
moy_NPERS=df_tcmmen[["NPERS"]].mean()
print(moy_NPERS)


pond_menC    2.925310e+07
dtype: float64
NPERS    2.292007
dtype: float64


In [32]:
#Type de ménage au sens du TCM : "TYPMEN5",
#Diplôme le plus élevé de la PR (code regroupé) : "DIPDETPR",
#Catégorie socioprofessionnelle du ménage : "CSMEN",
#Département de résidence : "DEP_RES",
#Région de la commune de résidence : "REG_res"
#variables de localisation...

nom_col_quanti=["quartile_rev","decile_rev"]

for col in nom_col_quanti :
    print("Résumé statistique pour la variable", col)
    print(df_tcmmen[col].describe())
    
print(df_tcmmen["quartile_rev"].head())


Résumé statistique pour la variable quartile_rev
count    13825.000000
mean         2.612658
std          1.115079
min          1.000000
25%          2.000000
50%          3.000000
75%          4.000000
max          4.000000
Name: quartile_rev, dtype: float64
Résumé statistique pour la variable decile_rev
count    13825.000000
mean         5.750741
std          2.872370
min          1.000000
25%          3.000000
50%          6.000000
75%          8.000000
max         10.000000
Name: decile_rev, dtype: float64
0    3
1    4
2    2
3    4
4    3
Name: quartile_rev, dtype: int64


    2.3 Analyse de la base men : 

In [6]:
#Affiche table q_menage
print(df_men.head())

       IDENT_MEN    pond_menC  JNBVEH  JNBVHS  JNBVPHS  JNBCCHS  JNBVULHS  \
0  1100000114000  1842.977077       1     2.0      NaN      NaN       NaN   
1  1100000115000  1948.382441       2     2.0      NaN      NaN       NaN   
2  1100000116000   833.453670       1     2.0      NaN      NaN       NaN   
3  1100000124000  1698.142782       1     2.0      NaN      NaN       NaN   
4  1100000125000  1994.633496       2     2.0      NaN      NaN       NaN   

   JNBMOTO  JNBCYCLO  JNBAUTVEH  ...  JUTILMOTO  JUTILCYCLO  JUTILAUTVEH  \
0        0         0          0  ...        NaN         NaN          NaN   
1        0         0          0  ...        NaN         NaN          NaN   
2        0         0          0  ...        NaN         NaN          NaN   
3        2         0          0  ...        2.0         NaN          NaN   
4        0         0          0  ...        NaN         NaN          NaN   

   JUTILVELOAD  JUTILVELOENF  JVOLVELO  JPLUSVELO  JNBKVM  JUTIL2RM  BLOGDIST  
